In [ ]:
# Import necessary libraries
#import PyPDF2
#import re
#import nltk
#from nltk.tokenize import word_tokenize
#from nltk.tag import pos_tag
#from nltk.chunk import ne_chunk
#import fitz  # PyMuPDF

# PDF Extractor
import tabula
# Data Manipulation
import pandas as pd
import numpy as np
# Dashboarding
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [ ]:
# Ensure necessary NLTK models are downloaded
#nltk.download('punkt')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

In [ ]:
def extract_and_combine_tables(pdf_path, page_numbers, new_header):
    combined_table = pd.DataFrame()
    
    for page in page_numbers:
        # Extract table from each page
        tables = tabula.read_pdf(pdf_path, pages=page, multiple_tables=True, lattice=True)
        
        if tables:
            df = tables[0]
            # Delete the first two rows from each page
            df = df.iloc[2:]
            combined_table = pd.concat([combined_table, df], ignore_index=True)
    # Keep only the first 5 columns
    combined_table = combined_table.iloc[:, :5]  

    # Set the new header for the combined table
    combined_table.columns = new_header
    
    return combined_table

# Specify your PDF path
pdf_path = '/Users/mohjaiswal/Desktop/ArthyaInvest/dvdfdf.pdf'
pages = [16, 17]  # Pages to extract
new_header = [
    "Balance Sheet", 
    "As at September 30, 2020", 
    "As at March 31, 2020", 
    "As at March 31, 2019", 
    "As at March 31, 2018"
]

# Call the function and assign the result to a variable
df = extract_and_combine_tables(pdf_path, pages, new_header)

# Display the DataFrame
df


In [ ]:
# Assuming 'df' is your DataFrame
# Clean and convert columns
for column in df.columns[1:]:  # Columns after the index
    # Remove unwanted characters (e.g., '$', commas) from the column
    df[column] = df[column].replace('[₹(),]', '', regex=True).replace(' ', '')

    # Convert to numeric, with custom handling for non-numeric data
    df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0)  # Replace NaN with 0

# View the DataFrame
df

In [ ]:
# Copy the DataFrame
indexed_df = df.copy()

# Set 'Balance Sheet' as the new index for the copied DataFrame
indexed_df.set_index('Balance Sheet', inplace=True)

# Now, indexed_df will have 'Balance Sheet' as its index
# Display the indexed DataFrame to verify the changes
# Set the maximum number of rows to display
pd.set_option('display.max_rows', 62)

# Now when you display the DataFrame, all 62 rows will be shown
indexed_df

In [ ]:
indexed_df.info()

In [ ]:
# Define the number of years for CAGR calculation
years = 2  # Adjust this based on your specific data

# Your existing code for calculations
def calculate_change(current, previous):
    if pd.notna(current) and pd.notna(previous) and previous != 0:
        return (current - previous) / previous * 100
    else:
        return 0

indexed_df['YoY 2019-2020'] = indexed_df.apply(lambda row: calculate_change(row[1], row[2]), axis=1)
indexed_df['YoY 2018-2019'] = indexed_df.apply(lambda row: calculate_change(row[2], row[3]), axis=1)
indexed_df['CAGR'] = indexed_df.apply(lambda row: ((row[1] / row[3]) ** (1 / years) - 1) * 100 if pd.notna(row[1]) and pd.notna(row[3]) and row[3] != 0 else 0, axis=1)

# Display the updated DataFrame
indexed_df

In [ ]:
indexed_df.info()

## Dashboarding 

In [ ]:
print(df.head())
print(df.columns)

In [ ]:
df.set_index('Balance Sheet', inplace=True)

In [ ]:
# Now you can access the value for 'Total current assets' using `.at` accessor
current_assets = df.at['Total current assets', 'As at September 30, 2020']

# Printing the value to verify
print(current_assets)

### DashBoarding
---
The terminology you've provided corresponds to the typical line items found in a balance sheet. To adjust the financial ratios using the given terminology, we need to consider how each line item relates to the common financial metrics. Here are the formulas adapted with the provided terminology:

1. **Current Ratio**:
   $$ \text{Current Ratio} = \frac{\text{Total Current Assets}}{\text{Total Current Liabilities}} $$

2. **Quick Ratio**:
   $$ \text{Quick Ratio} = \frac{\text{Total Current Assets} - \text{Inventories (if listed)}}{\text{Total Current Liabilities}} $$
   Note: If inventories are not listed separately, they might be part of 'Other current assets' or not applicable.

3. **Debt-to-Equity Ratio**:
   $$ \text{Debt-to-Equity Ratio} = \frac{\text{Total Non-current Liabilities} + \text{Total Current Liabilities}}{\text{Equity Attributable to Equity Holders of the Company} + \text{Non-controlling Interests}} $$

4. **Return on Equity (ROE)**:
   $$ \text{ROE} = \frac{\text{Net Income}}{\text{Equity Attributable to Equity Holders of the Company} + \text{Non-controlling Interests}} $$

5. **Return on Assets (ROA)**:
   $$ \text{ROA} = \frac{\text{Net Income}}{\text{Total Assets}} $$

6. **Asset Turnover Ratio**:
   $$ \text{Asset Turnover Ratio} = \frac{\text{Net Sales}}{\text{Total Assets}} $$

7. **Inventory Turnover Ratio**:
   (If inventory is part of 'Other current assets', you would need to extract the inventory amount to calculate this ratio.)
   $$ \text{Inventory Turnover Ratio} = \frac{\text{Cost of Goods Sold}}{\text{Inventory}} $$

8. **Accounts Receivable Turnover Ratio**:
   $$ \text{Accounts Receivable Turnover} = \frac{\text{Net Credit Sales}}{\text{Trade Receivables}} $$

9. **Net Working Capital**:
   $$ \text{Net Working Capital} = \text{Total Current Assets} - \text{Total Current Liabilities} $$

10. **Leverage Ratios**:
    $$ \text{Times Interest Earned} = \frac{\text{EBIT}}{\text{Interest Expense from Borrowings}} $$

**Note**:  Some of the specific line items you might need for these calculations, such as 'Net Income', 'Net Sales', 'Cost of Goods Sold', and 'EBIT' (Earnings Before Interest and Taxes), are not listed in the terminology provided. These would typically be found on the income statement. Additionally, some ratios like Inventory Turnover and Accounts Receivable Turnover depend on more detailed information than what is provided in the balance sheet summary.


In [ ]:
# Initialize the Dash application
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div(children=[
    html.H1(children='Balance Sheet Dashboard'),

    # Dropdown to select the year for which the metrics are to be displayed
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in df.columns],
        value=df.columns[0]
    ),
    
    # Div to hold the financial metrics
    html.Div(id='financial-metrics'),

    # Any additional graphs or tables can be added here
    # ...
])

# Define the callback to update the financial metrics
@app.callback(
    Output('financial-metrics', 'children'),
    [Input('year-dropdown', 'value')]
)
def update_financial_metrics(selected_year):
    # Initialize a list to hold the metrics elements
    metrics_list = [html.H3(f"Financial Metrics for {selected_year}")]
    
    # Helper function to safely calculate ratios
    def calculate_ratio(numerator, denominator):
        try:
            return numerator / denominator if denominator != 0 else "Undefined (Denominator is 0)"
        except (TypeError, ValueError):
            return "Data not available"

    # Helper function to get value from DataFrame safely
    def get_value(label, default=0):
        return df.loc[label, selected_year] if label in df.index else default
    
    # Calculate each ratio, handling missing data or errors
    current_ratio = calculate_ratio(get_value('Total current assets'), get_value('Total current liabilities'))
    quick_ratio = calculate_ratio(get_value('Total current assets') - get_value('Inventories'), get_value('Total current liabilities'))
    total_equity = get_value('Equity attributable to equity holders of the Company') + get_value('Non-controlling interests')
    total_liabilities = get_value('Total non-current liabilities') + get_value('Total current liabilities')
    debt_to_equity_ratio = calculate_ratio(total_liabilities, total_equity)
    net_income = get_value('Net Income')
    roe = calculate_ratio(net_income, total_equity)
    total_assets = get_value('Total assets')
    roa = calculate_ratio(net_income, total_assets)
    net_sales = get_value('Net Sales')
    asset_turnover_ratio = calculate_ratio(net_sales, total_assets)
    net_working_capital = get_value('Total current assets') - get_value('Total current liabilities')
    
    # Add the calculated metrics to the metrics list
    metrics_list.append(html.P(f"Current Ratio: {current_ratio}"))
    metrics_list.append(html.P(f"Quick Ratio: {quick_ratio}"))
    metrics_list.append(html.P(f"Debt to Equity Ratio: {debt_to_equity_ratio}"))
    metrics_list.append(html.P(f"Return on Equity (ROE): {roe}%"))
    metrics_list.append(html.P(f"Return on Assets (ROA): {roa}%"))
    metrics_list.append(html.P(f"Asset Turnover Ratio: {asset_turnover_ratio}"))
    metrics_list.append(html.P(f"Net Working Capital: {net_working_capital}"))

    return metrics_list

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
# Note: This is just a gimmick for making the dataframe easy on the eyes where all columns containing 'Total' are made bold. Usually at this stage, the df will be exported to excel 
# Assuming indexed_df is your DataFrame
# Temporarily reset the index for styling
temp_df = indexed_df.reset_index()

# Define a bold styling function
def bold_if_total(row):
    if 'Total' in row.to_string():
        return ['font-weight: bold'] * len(row)
    else:
        return [''] * len(row)

# Apply the styling
bold_df = temp_df.style.apply(bold_if_total, axis=1)

# Display the styled DataFrame
bold_df